In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [7]
commitnumber = de5d43e
molecule = {'o3': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3
  ng_adju = [0, 0]
  ng_refs = [2, 5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.6, 1.75), (1.55, 1.66, 1.7, 1.75, 1.8)]
  wgt = [(0.35, 0.6), (0.5, 0.55, 0.7, 0.9, 1.0)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.028373                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.016447
0.000750      2.0 -0.029882                  2.0 -0.016911
0.001052      3.0 -0.029822                  3.0 -0.015836
0.001476      4.0 -0.025147                  4.0 -0.011373
0.002070      5.0 -0.015298                  5.0 -0.003150
0.002904      6.0  0.000418                  6.0  0.009390
0.004074      7.0  0.022764                  7.0  0.026823
0.005714      8.0  0.052554                  8.0  0.049769
0.008015      9.0  0.090594                  9.0  0.078922
0.011243     10.0  0.137567                 10.0  0.115039
0.015771     11.0  0.194546                 11.0  0.159072
0.022122     12.0  0.262192                 12.0  0.211718
0.031031     13.0  0.340991                 13.0  0.273636
0.043528     14.0  0.460107                 14.0  0.369228
0.061057     15.0  0.637544                 15.0  0.515364
0.085645     16.0  0.842350                 16.0  0.690519
0.120136     17.0  1.059467                 17.0  0.886223
0.168516     18.0  1.281938                 18.0  1.100459
0.236378     19.0  1.502461                 19.0  1.329192
0.331549     20.0  1.717483                 20.0  1.566793
0.465100     21.0  1.919230                 21.0  1.794661
0.652400     22.0  1.966398                 22.0  1.897794
0.915100     23.0  1.657827                 23.0  1.670714
1.283650     24.0  1.248061                 24.0  1.280581
1.800600     25.0  0.925249                 25.0  0.934962
2.525700     26.0  0.647655                 26.0  0.650581
3.542800     27.0  0.425198                 27.0  0.435601
4.969550     28.0  0.284234                 28.0  0.299539
6.970850     29.0  0.210428                 29.0  0.231478
9.778100     30.0  0.157681                 30.0  0.178660
13.715850    31.0  0.112014                 31.0  0.128738
19.239350    32.0  0.065161                 32.0  0.076287
26.987250    33.0  0.036001                 33.0  0.041985
37.855300    34.0  0.022802                 34.0  0.024210
53.100050    35.0 -0.001029                 35.0 -0.002608
73.887500    36.0 -0.031344                 36.0 -0.036492
97.662500    37.0 -0.057120                 37.0 -0.067870
121.437500   38.0 -0.075912                 38.0 -0.091538
145.212500   39.0 -0.090697                 39.0 -0.109165
168.987500   40.0 -0.103255                 40.0 -0.122216
192.762500   41.0 -0.103060                 41.0 -0.117950
216.537500   42.0 -0.093096                 42.0 -0.101903
240.312500   43.0 -0.082666                 43.0 -0.086145
264.087500   44.0 -0.069603                 44.0 -0.069012
287.862500   45.0 -0.055837                 45.0 -0.052883
311.637500   46.0 -0.040852                 46.0 -0.037383
335.412500   47.0 -0.028025                 47.0 -0.025082
359.187500   48.0 -0.020243                 48.0 -0.017786
382.962500   49.0 -0.014679                 49.0 -0.012625
406.737500   50.0 -0.010093                 50.0 -0.008452
430.512500   51.0 -0.006359                 51.0 -0.005128
454.287500   52.0 -0.003683                 52.0 -0.002775
478.062500   53.0 -0.001731                 53.0 -0.001044
501.837500   54.0 -0.000020                 54.0  0.000471
525.612500   55.0  0.001510                 55.0  0.001822
549.387500   56.0  0.002876                 56.0  0.003025
573.162500   57.0  0.004074                 57.0  0.004078
596.937500   58.0  0.005114                 58.0  0.004989
620.712500   59.0  0.006013                 59.0  0.005776
644.487500   60.0  0.006765                 60.0  0.006433
668.262500   61.0  0.007369                 61.0  0.006957
692.037500   62.0  0.007866                 62.0  0.007390
715.812500   63.0  0.008279                 63.0  0.007749
739.587500   64.0  0.008593                 64.0  0.008020
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -10.995326  0.000000 -10.995326        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -10.705945  0.000002   
6.244000e-04 2     -10.995330  0.000002 -10.995328 -10.705947  0.000004   
8.759000e-04 3     -10.995332  0.000003 -10.995329 -10.705949  0.000004   
1.228600e-03 4     -10.995336  0.000005 -10.995330 -10.705951  0.000006   
1.723400e-03 5     -10.995341  0.000010 -10.995332 -10.705954  0.000009   
2.417400e-03 6     -10.995350  0.000017 -10.995333 -10.705960  0.000014   
3.390900e-03 7     -10.995363  0.000030 -10.995333 -10.705968  0.000023   
4.756500e-03 8     -10.995382  0.000052 -10.995329 -10.705980  0.000039   
6.672000e-03 9     -10.995408  0.000091 -10.995317 -10.705997  0.000067   
9.358900e-03 10    -10.995445  0.000157 -10.995288 -10.706019  0.000115   
1.312780e-02 11    -10.995493  0.000266 -10.995227 -10.706048  0.000195   
1.841450e-02 12    -10.995554  0.000449 -10.995105 -10.706082  0.000329   
2.583020e-02 13    -10.995625  0.000750 -10.994875 -10.706117  0.000550   
3.623230e-02 14    -10.995695  0.001241 -10.994455 -10.706144  0.000914   
5.082340e-02 15    -10.995756  0.002096 -10.993659 -10.706146  0.001555   
7.129060e-02 16    -10.995783  0.003670 -10.992114 -10.706089  0.002747   
1.000000e-01 17    -10.995710  0.006461 -10.989249 -10.705895  0.004902   
1.402710e-01 18    -10.995379  0.011184 -10.984195 -10.705401  0.008636   
1.967600e-01 19    -10.994479  0.018863 -10.975617 -10.704299  0.014898   
2.759970e-01 20    -10.992413  0.030898 -10.961514 -10.702012  0.025088   
3.871000e-01 21    -10.988007  0.049096 -10.938911 -10.697458  0.041157   
5.431000e-01 22    -10.978972  0.075527 -10.903445 -10.688678  0.065545   
7.617000e-01 23    -10.963519  0.110994 -10.852525 -10.674072  0.100087   
1.068500e+00 24    -10.943219  0.150943 -10.792276 -10.654750  0.141489   
1.498800e+00 25    -10.919479  0.190820 -10.728659 -10.631795  0.183815   
2.102400e+00 26    -10.892510  0.230007 -10.662503 -10.605291  0.224168   
2.949000e+00 27    -10.865238  0.267685 -10.597553 -10.578180  0.262308   
4.136600e+00 28    -10.842816  0.305080 -10.537736 -10.555477  0.300892   
5.802500e+00 29    -10.828890  0.347244 -10.481646 -10.540744  0.345275   
8.139200e+00 30    -10.822696  0.399296 -10.423400 -10.533324  0.401935   
1.141700e+01 31    -10.825121  0.462945 -10.362176 -10.534250  0.472237   
1.601470e+01 32    -10.843781  0.542612 -10.301169 -10.551306  0.559415   
2.246400e+01 33    -10.897376  0.645987 -10.251389 -10.603380  0.669776   
3.151050e+01 34    -11.004191  0.791382 -10.212809 -10.709846  0.821239   
4.420010e+01 35    -11.171191  0.992657 -10.178534 -10.879593  1.027381   
6.200000e+01 36    -11.417520  1.236816 -10.180704 -11.137188  1.279475   
8.577500e+01 37    -11.762437  1.493457 -10.268980 -11.511542  1.551047   
1.095500e+02 38    -12.108055  1.678208 -10.429847 -11.902159  1.750500   
1.333250e+02 39    -12.446402  1.802762 -10.643640 -12.297381  1.887894   
1.571000e+02 40    -12.789607  1.890535 -10.899072 -12.703758  1.986797   
1.808750e+02 41    -13.148024  1.958152 -11.189873 -13.125270  2.064074   
2.046500e+02 42    -13.487858  2.007735 -11.480123 -13.514839  2.121424   
2.284250e+02 43    -13.785511  2.043199 -11.742311 -13.843305  2.162871   
2.522000e+02 44    -14.044750  2.069625 -11.975125 -14.117081  2.194010   
2.759750e+02 45    -14.260286  2.089138 -12.171148 -14.334596  2.217145   
2.997500e+02 46    -14.432440  2.104037 -12.328403 -14.501082  2.234681   
3.235250e+02 47    -14.561606  2.118150 -12.443456 -14.621851  2.250157   
3.473000e+02 48    -14.656464  2.134081 -12.522383 -14.708561  2.266220   
3.710750e+02 49    -14.732045  2.152652 -12.579393 -14.776543  2.284105   
3.948500e+02 50    -14.793866  2.173133 -12.620733 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')